# LAPD & LA City aircraft fleet analysis

This notebook demonstrates how to use the `hangarbay` Python API to analyze aircraft fleets. We'll look at helicopters and aircraft operated by the Los Angeles Police Department and LA City Fire Department.

## Setup

First, install and import hangarbay:


In [1]:
# Install (only needed once)
# !pip install hangarbay

import hangarbay as hb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style('whitegrid')


## Load FAA data

This downloads and processes the FAA aircraft registry (~400MB, one-time operation).
Data is stored in `~/.hangarbay/data/` so it's accessible from any notebook.


In [2]:
# One-time setup - downloads and processes FAA data
# Use quiet=True to suppress progress output in notebooks
hb.load_data(quiet=True)

In [3]:
# Check data status
info = hb.status()
print(f"Data snapshot: {info['snapshot_date']}")
print(f"Data age: {info['age_days']} days")
print(f"Total aircraft: {info['aircraft_count']:}")

Data snapshot: 2025-11-09
Data age: 0 days
Total aircraft: 307793


## LAPD air support division

Let's find all aircraft owned by LAPD. Since the FAA data uses various forms of the name, we'll use wildcard search with the `|` operator for OR logic.


In [4]:
# Search for LAPD aircraft using wildcard OR logic
lapd = hb.fleet("LAPD|Los Angeles Police")

print(f"Found {len(lapd)} aircraft registered to LAPD\n")
lapd.head()

Found 16 aircraft registered to LAPD



,n_number,maker,model,year_mfr,reg_status,owner_name,city,state
0,213PF,AIRBUS HELICOPTERS INC,AS350B3,2017,Valid,LAPD AIR SUPPORT DIVISION,LOS ANGELES,CA
1,21844,BELL,412EP,1999,Valid,LOS ANGELES POLICE DEPT AIR SUPPORT DIVISION,LOS ANGELES,CA
2,221LA,AIRBUS HELICOPTERS INC,AS350B3,2014,Valid,LAPD AIR SUPPORT DIVISION,LOS ANGELES,CA
3,223LA,AIRBUS HELICOPTERS INC,AS350B3,2014,Valid,LAPD AIR SUPPORT DIVISION,LOS ANGELES,CA
4,224LA,AIRBUS HELICOPTERS INC,AS350B3,2022,Valid,LAPD AIR SUPPORT DIVISION,LOS ANGELES,CA


### Fleet summary


In [5]:
# Show owner variations
print("Owner name variations in FAA data:")
print(lapd['owner_name'].unique())

# Fleet statistics
print(f"\nTotal aircraft: {len(lapd)}")
print(f"Valid registrations: {(lapd['reg_status'] == 'Valid').sum()}")
print(f"\nManufacturers:")
print(lapd['maker'].value_counts())


Owner name variations in FAA data:
['LAPD AIR SUPPORT DIVISION'
 'LOS ANGELES POLICE DEPT AIR SUPPORT DIVISION'
 'LOS ANGELES POLICE DEPARTMENT']

Total aircraft: 16
Valid registrations: 16

Manufacturers:
maker
AIRBUS HELICOPTERS INC     9
EUROCOPTER                 4
BELL                       1
AMERICAN EUROCOPTER LLC    1
BEECH                      1
Name: count, dtype: int64


### Aircraft details


In [6]:
# Display full fleet with key details
lapd_display = lapd[['n_number', 'maker', 'model', 'year_mfr', 'reg_status']].copy()
lapd_display['n_number'] = 'N' + lapd_display['n_number'].astype(str)
lapd_display.columns = ['N-Number', 'Manufacturer', 'Model', 'Year', 'Status']
lapd_display.sort_values('N-Number')


,N-Number,Manufacturer,Model,Year,Status
0,N213PF,AIRBUS HELICOPTERS INC,AS350B3,2017,Valid
1,N21844,BELL,412EP,1999,Valid
2,N221LA,AIRBUS HELICOPTERS INC,AS350B3,2014,Valid
3,N223LA,AIRBUS HELICOPTERS INC,AS350B3,2014,Valid
4,N224LA,AIRBUS HELICOPTERS INC,AS350B3,2022,Valid
5,N225LA,AIRBUS HELICOPTERS INC,AS350B3,2019,Valid
6,N226LA,AIRBUS HELICOPTERS INC,AS350B3,2019,Valid
7,N230LA,AIRBUS HELICOPTERS INC,AS350B3,2020,Valid
8,N267LA,AIRBUS HELICOPTERS INC,AS350B3,2017,Valid
9,N472LA,AIRBUS HELICOPTERS INC,AS350B3,2018,Valid


## LA City government aircraft

Let's search for City of Los Angeles aircraft. This includes Fire Department, Water & Power and other city agencies.


In [7]:
# Search for City of Los Angeles aircraft
la_city = hb.fleet("City of Los Angeles", state="CA")

print(f"Found {len(la_city)} aircraft registered to City of Los Angeles\n")

if len(la_city) > 0:
    # Show the different city departments
    print("City Departments:")
    for owner in sorted(la_city['owner_name'].unique()):
        count = (la_city['owner_name'] == owner).sum()
        print(f"  • {owner}: {count} aircraft")
    
    print("\n")
    
    # Analyze by N-number suffix (indicates department)
    print("N-number patterns reveal department assignments:")
    print("  • N___FD: Fire Department helicopters (304FD, 305FD, etc.)")
    print("  • N___WP: Dept of Water & Power helicopters (703WP, 704WP, etc.)")
    print("  • N___LA: Other city aircraft, possibly LAPD civilian use")
    print("  • Other: Miscellaneous city aircraft")
    print()
    
    # Display all city aircraft
    city_display = la_city[['n_number', 'maker', 'model', 'year_mfr', 'owner_name']].copy()
    city_display['n_number'] = 'N' + city_display['n_number'].astype(str)
    city_display.columns = ['N-Number', 'Manufacturer', 'Model', 'Year', 'Owner']
    display(city_display.sort_values('N-Number'))

Found 13 aircraft registered to City of Los Angeles

City Departments:
  • CITY OF LOS ANGELES: 11 aircraft
  • CITY OF LOS ANGELES DEPARTMENT OF WATER & POWER: 1 aircraft
  • CITY OF LOS ANGELES DEPT OF WATER & POWER: 1 aircraft


N-number patterns reveal department assignments:
  • N___FD: Fire Department helicopters (304FD, 305FD, etc.)
  • N___WP: Dept of Water & Power helicopters (703WP, 704WP, etc.)
  • N___LA: Other city aircraft, possibly LAPD civilian use
  • Other: Miscellaneous city aircraft



,N-Number,Manufacturer,Model,Year,Owner
0,N228LA,EUROCOPTER,AS 350 B2,2000,CITY OF LOS ANGELES
1,N229LA,EUROCOPTER,AS 350 B2,2000,CITY OF LOS ANGELES
2,N232LA,AIRBUS HELICOPTERS INC,AS350B3,2023,CITY OF LOS ANGELES
3,N233LA,AIRBUS HELICOPTERS INC,AS350B3,2024,CITY OF LOS ANGELES
4,N304FD,AGUSTAWESTLAND PHILADELPHIA,AW139,2017,CITY OF LOS ANGELES
5,N305FD,AGUSTA SPA,AW139,2008,CITY OF LOS ANGELES
6,N306FD,BELL TEXTRON CANADA LTD,505,2023,CITY OF LOS ANGELES
7,N307FD,BELL TEXTRON CANADA LTD,505,2022,CITY OF LOS ANGELES
8,N703WP,AIRBUS HELICOPTERS INC,AS350B3,2017,CITY OF LOS ANGELES
9,N704WP,AIRBUS HELICOPTERS INC,AS350B3,2017,CITY OF LOS ANGELES


In [8]:
# Combine LAPD and other city aircraft for complete municipal fleet
all_city = pd.concat([lapd, la_city]).drop_duplicates(subset=['n_number'])

print(f"=== Complete Los Angeles Municipal Fleet ===\n")
print(f"Total aircraft: {len(all_city)}\n")

# Summary by owner/agency
print("Fleet by Agency:")
agency_counts = all_city.groupby('owner_name').size().sort_values(ascending=False)
for owner, count in agency_counts.items():
    print(f"  • {owner}: {count} aircraft")

print("\n")

# Breakdown by manufacturer
print("Top Manufacturers:")
maker_counts = all_city['maker'].value_counts().head(5)
for maker, count in maker_counts.items():
    print(f"  • {maker}: {count} aircraft")

print("\n")

# Age analysis
valid_years = all_city[all_city['year_mfr'].notna()]
if len(valid_years) > 0:
    avg_age = 2025 - valid_years['year_mfr'].mean()
    oldest = int(valid_years['year_mfr'].min())
    newest = int(valid_years['year_mfr'].max())
    print(f"Fleet Age Analysis:")
    print(f"  • Average age: {avg_age:.1f} years")
    print(f"  • Oldest: {oldest} ({2025-oldest} years old)")
    print(f"  • Newest: {newest} ({2025-newest} years old)")

=== Complete Los Angeles Municipal Fleet ===

Total aircraft: 29

Fleet by Agency:
  • LAPD AIR SUPPORT DIVISION: 13 aircraft
  • CITY OF LOS ANGELES: 11 aircraft
  • LOS ANGELES POLICE DEPT AIR SUPPORT DIVISION: 2 aircraft
  • CITY OF LOS ANGELES DEPARTMENT OF WATER & POWER: 1 aircraft
  • CITY OF LOS ANGELES DEPT OF WATER & POWER: 1 aircraft
  • LOS ANGELES POLICE DEPARTMENT: 1 aircraft


Top Manufacturers:
  • AIRBUS HELICOPTERS INC: 13 aircraft
  • EUROCOPTER: 6 aircraft
  • BELL TEXTRON CANADA LTD: 4 aircraft
  • BELL: 2 aircraft
  • AMERICAN EUROCOPTER LLC: 1 aircraft


Fleet Age Analysis:
  • Average age: 13.6 years
  • Oldest: 1963 (62 years old)
  • Newest: 2024 (1 years old)


## Individual aircraft lookup

For detailed information on a specific aircraft, use the `search()` function:


In [9]:
# Look up a specific LAPD helicopter
if len(lapd) > 0:
    # Get the first N-number from our LAPD fleet
    example_n = lapd.iloc[0]['n_number']
    
    print(f"Detailed lookup for N{example_n}:\n")
    aircraft = hb.search(example_n)
    
    # Display key fields
    display_fields = ['n_number', 'maker', 'model', 'year_mfr', 'reg_status', 
                      'owner_name', 'city', 'state', 'reg_expiration']
    for col in display_fields:
        if col in aircraft.columns:
            value = aircraft.iloc[0][col]
            if pd.notna(value) and value != '':
                print(f"{col}: {value}")


Detailed lookup for N213PF:

n_number: 213PF
maker: AIRBUS HELICOPTERS INC
model: AS350B3
year_mfr: 2017
reg_status: Valid
owner_name: LAPD AIR SUPPORT DIVISION
city: LOS ANGELES
state: CA
reg_expiration: 2028-08-31 00:00:00


## Export data

Save the LAPD fleet data for further analysis:


In [10]:
# Export to CSV
if len(lapd) > 0:
    output_file = 'lapd_fleet.csv'
    lapd.to_csv(output_file, index=False)
    print(f"✓ Exported {len(lapd)} aircraft to {output_file}")


✓ Exported 16 aircraft to lapd_fleet.csv
